Formerly called minreverse

In [1]:
import gurobipy as gp
from gurobipy import GRB

In [2]:
from compute import Var, eqvar, Evaluable, ureg, coupled_run, buildidpvars, Impcomp
from inputresolver import reassigneq, eqvars, eqsonly, default_out, mdf_order, getdofs, idx_eqlist, getallvars, invert_edges, var_matched_cons
from inputresolver import reassign
from compute_utils import get_outputs, check_eqs
from representations import bipartite_repr, drawbipartite, digraph_repr
from utils import invmap
import openmdao.api as om
import sympy as sp
import networkx as nx
from functools import reduce

Unable to import mpi4py. Parallel processing unavailable.
Unable to import petsc4py. Parallel processing unavailable.
Unable to import petsc4py. Parallel processing unavailable.


In [3]:
class Impcomp(om.ImplicitComponent):
    def initialize(self):
        self.options.declare('equation')
        self.options.declare('output_name')
        
    def setup(self):
        equation = self.options['equation']
        output_name = self.options['output_name']
        self.add_output(output_name)
        original_inputs = [inp for inp in equation.input_names if inp != output_name]
        for name in original_inputs:
            self.add_input(name, val=1.) # add them in the order we lambdify
        self.declare_partials(output_name, equation.input_names)

    def apply_nonlinear(self, inputs, outputs, residuals):
        equation = self.options['equation']
        output_name = self.options['output_name']
        residuals[output_name] = equation.evaldict({**inputs, **outputs})
        
    def linearize(self, inputs, outputs, partials):
        equation = self.options['equation']
        output_name = self.options['output_name']
        J = equation.graddict({**inputs, **outputs})
        for idx, input_name in enumerate(equation.input_names):
            partials[output_name, input_name] = J[idx]

In [71]:
eq = eqlist[0]
eq

(b, c + 1)

In [72]:
ev = Evaluable.fromsympy(eq[0]-eq[1])

In [73]:
ev.input_names

['c', 'b']

In [75]:
prob = om.Problem()
model = prob.model

model.add_subsystem('comp', Impcomp(equation = ev, output_name='b'), promotes=['*'])
model.nonlinear_solver = om.NewtonSolver(solve_subsystems=False)
model.linear_solver = om.ScipyKrylov()

prob.setup()
prob.set_val('c', 3.)
prob.run_model()

print(prob.get_val('comp.b'))

NL: Newton Converged in 1 iterations
[4.]


In [3]:
a = Var('a')
b = Var('b')
c = Var('c')
d = Var('d')

In [4]:
eqlist = [(b,c+1),(a,b-2),(a,2*d+3)]

In [5]:
eqs = idx_eqlist(eqlist)
eqv = eqvars(eqs)

In [51]:
eqs

{0: (b, c + 1), 1: (a, b - 2), 2: (a, 2*d + 3)}

In [8]:
eqv

{0: {b, c}, 1: {a, b}, 2: {a, d}}

In [9]:
dout = default_out(eqs)

In [10]:
dout

{0: b, 1: a, 2: a}

In [16]:
G, edges_original = bipartite_repr(eqv)
vrs = getallvars(eqs)
eqns = eqs.keys()
edges = invert_edges(edges_original)

In [40]:
not_input = []
n_eqs = len(eqns)

In [45]:
m = gp.Model('minrevassign')
m.setParam('OutputFlag', False )
x = m.addVars(edges, name="assign", vtype=GRB.BINARY)

In [46]:
# Matching eqs:
fcons = m.addConstrs((x.sum(j,'*') == 1 for j in eqns), name='equations')
varcons = m.addConstrs((var_matched_cons(x, j, not_input) for j in vrs), name='variables')
m.setObjective(gp.quicksum([x[(key, var)] for key,var in dout.items()]), GRB.MAXIMIZE)
#m.addConstr(gp.quicksum(x[(r,j)] for r,j in sol)<= n_eqs-1) 

In [47]:
m.optimize()

In [48]:
m.objVal

2.0

In [49]:
sol = [(r,j) for (r, j) in edges if x[r,j].x>1e-6]

In [52]:
sol_outset = {key:val for key,val in sol}

In [53]:
sol_outset

{0: b, 1: a, 2: d}

## Coffee thesis example

In [4]:
ureg.define('USD = [currency] ')

In [5]:
# the force_input, force_output tags are for other algorithms, so probably makes little sense to include them here
IFOV = Var('IFOV', 7.272e-5, 'rad') #force_output=True
r = Var('r', 9.257e19, 'm')
l = Var('lambda', 5e-7, 'm')
d = Var('d', 3e-5, 'm')
Q = Var('Q', 1.1)
rho = Var('rho', 1.22, never_output=True) #force_input=True
k = Var('k', 1.381e-23, 'J/K', never_output=True)
T = Var('T', 5785, 'K')
h = Var('h', 6.626e-34, 'J*s', never_output=True)
c = Var('c', 2.998e8, 'm/s', never_output=True)
tau = Var('tau', 1)
dl = Var('Delta_lambda', 2e-6, 'm')
R = Var('R', 6.96e8, 'm')
dV = Var('Delta_V', 0, 'm/s')
g = Var('g', 9.8, 'm/s^2', never_output=True)
isp = Var('I_sp', 450, 's')
ct = Var('c_t', 1163, 'USD/kg')
ms = Var('m_s', 1175, 'kg/m^2')
QE = Var('Q_E', 0.5)
Nr = Var('N_r', 25)
tau0 = Var('tau_0', 0.75)
Ti = Var('T_i', 30, 's')
a = Var('alpha', 50e3, 'USD/m^3')
SNR = Var('SNR', 10, never_output=True)
CT = Var('C_T', 1e6, 'USD', never_output=True)

In [6]:
Y, eq1 = eqvar('Y', IFOV*r, 'm')
f, eq2 = eqvar('f', r*d/Y, 'm')
D, eq3 = eqvar('D', 2*rho*l*Q*f/d, 'm')
tr, eq4 = eqvar('theta_r', rho*l/D, 'rad')
mt, eq5 = eqvar('mt', ms*D**2, 'kg')
mi, eq6 = eqvar('m_i', mt*sp.exp(dV/(g*isp)), 'kg')
CD, eq7 = eqvar('C_D', a*D**3, 'USD')
eq8 = (CT, CD+ct*mi)
Hl, eq9 = eqvar('H_lambda', 2*sp.pi*h*c**2/l**5*1/(sp.exp(c*h/(k*T*l))-1), 'W/m^3')
L, eq10 = eqvar('L', 1/4*R**2*Hl*tau*dl, 'W/sr')
Pin, eq11 = eqvar('P_in', sp.pi*(D/(2*r))**2*L, 'W')
Hi, eq12 = eqvar('H_i', Pin*tau0*Ti, 'J')
Np, eq13 = eqvar('N_p', Hi*l/(h*c))
#SNR, eq14 = eqvar('SNR', Np*QE/sp.sqrt(Nr**2+Np*QE))
eq14 = (SNR, Np*QE/sp.sqrt(Nr**2+Np*QE))

In [7]:
eqlist = [eq1, eq2, eq3, eq4, eq5, eq6, eq7, eq8, eq9, eq10, eq11, eq12, eq13, eq14]
eqs = idx_eqlist(eqlist)
eqv = eqvars(eqs)

In [8]:
dout = default_out(eqs)

In [122]:
outnotpossible = {
    13: {Nr, QE}
}

In [124]:
dout

{0: Y,
 1: f,
 2: D,
 3: theta_r,
 4: mt,
 5: m_i,
 6: C_D,
 7: C_T,
 8: H_lambda,
 9: L,
 10: P_in,
 11: H_i,
 12: N_p,
 13: SNR}

In [125]:
eqvcst = {key: {elt for elt in outsetposs if not elt.never_output and elt not in outnotpossible.get(key,[])}
        for key, outsetposs in eqv.items()}

In [126]:
G, edges_original = bipartite_repr(eqvcst)
vrs = getallvars(eqs)
eqns = eqs.keys()
edges = invert_edges(edges_original)

In [127]:
not_input = [IFOV, Ti]

In [128]:
n_eqs = len(eqns)

In [129]:
eqvcst

{0: {IFOV, Y, r},
 1: {Y, d, f, r},
 2: {D, Q, d, f, lambda},
 3: {D, lambda, theta_r},
 4: {D, m_s, mt},
 5: {Delta_V, I_sp, m_i, mt},
 6: {C_D, D, alpha},
 7: {C_D, c_t, m_i},
 8: {H_lambda, T, h, lambda},
 9: {Delta_lambda, H_lambda, L, R, tau},
 10: {D, L, P_in, r},
 11: {H_i, P_in, T_i, tau_0},
 12: {H_i, N_p, h, lambda},
 13: {N_p}}

In [130]:
m = gp.Model('minrevassign')
m.setParam('OutputFlag', False )
x = m.addVars(edges, name="assign", vtype=GRB.BINARY)
# Matching eqs:
fcons = m.addConstrs((x.sum(j,'*') == 1 for j in eqns), name='equations')
varcons = m.addConstrs((var_matched_cons(x, j, not_input) for j in vrs), name='variables')
m.setObjective(gp.quicksum([x[(key, var)] for key,var in dout.items() if var in eqvcst[key]]), GRB.MAXIMIZE)

In [131]:
#m.addConstr(gp.quicksum(x[(r,j)] for r,j in sol)<= n_eqs-1)
m.optimize()

In [132]:
m.objVal

9.0

In [133]:
sol = [(r,j) for (r, j) in edges if x[r,j].x>1e-6]
sol_outset = {key:val for key,val in sol}
sol_outset

{0: IFOV,
 1: f,
 2: D,
 3: theta_r,
 4: mt,
 5: m_i,
 6: C_D,
 7: c_t,
 8: H_lambda,
 9: L,
 10: P_in,
 11: T_i,
 12: h,
 13: N_p}

In [134]:
#new_eqs=reassign(eqs, dict(sol_outset))
#new_ins = getdofs(new_eqs)
d = dict(sol_outset)
#d = dout
order = mdf_order(eqv, d)

In [135]:
order

[13, 12, 8, 9, 1, 2, 10, 11, 6, 4, 5, 7, 3, 0]

In [136]:
ins = reduce(set.union, eqv.values()) - set(d.values())

## Debug

In [115]:
ev = Evaluable.fromsympy(eq14[0]-eq14[1])

In [116]:
evrt = Evaluable.fromsympy(eq14[1])

In [93]:
eq14

(SNR, N_p*Q_E/sqrt(N_p*Q_E + N_r**2))

In [81]:
evrt.input_names, Np.varval, QE.varval, Nr.varval

(['N_p', 'Q_E', 'N_r'], None, 0.5, 25)

In [92]:
eq14

(SNR, N_p*Q_E/sqrt(N_p*Q_E + N_r**2))

In [96]:
evrt.fx(2.45e-3, 0.5, 30)

4.0833305544009846e-05

In [66]:
ev.fx(2.45e-3, QE.varval, SNR.varval, 0.00001)

9.965000001428571

In [34]:
Np.varval

In [48]:
prob = om.Problem()
model = prob.model

buildidpvars({Np, QE, SNR}, model)
model.add_subsystem('comp', Impcomp(equation = ev, output_name='N_r'), promotes=['*'])
model.nonlinear_solver = om.NewtonSolver(solve_subsystems=False)
model.linear_solver = om.ScipyKrylov()

prob.setup()
prob.set_val('N_p', 2.45e-3)
prob.run_model()

print(prob.get_val('N_r'))

NL: NewtonSolver 'NL: Newton' on system '' failed to converge in 10 iterations.
[-1.14022952e+122]


In [44]:
model.list_inputs()

3 Input(s) in 'model'

varname  value    
-------  ---------
comp
  N_p    [0.00245]
  Q_E    [0.5]    
  SNR    [10.]    




[('comp.N_p', {'value': array([0.00245])}),
 ('comp.Q_E', {'value': array([0.5])}),
 ('comp.SNR', {'value': array([10.])})]

In [137]:
prob = om.Problem()
model = prob.model
buildidpvars(ins, model)
counter = coupled_run(eqs, order, (), model, model, d)
prob.setup()
prob.run_model()

eq13 SNR N_p*Q_E/sqrt(N_p*Q_E + N_r**2) N_p
eq12 N_p H_i*lambda/(c*h) h
eq8 H_lambda 2*pi*c**2*h/(lambda**5*(exp(c*h/(T*k*lambda)) - 1)) H_lambda
eq9 L 0.25*Delta_lambda*H_lambda*R**2*tau L
eq1 f d*r/Y f
eq2 D 2*Q*f*lambda*rho/d D
eq10 P_in pi*D**2*L/(4*r**2) P_in
eq11 H_i P_in*T_i*tau_0 T_i
eq6 C_D D**3*alpha C_D
eq4 mt D**2*m_s mt
eq5 m_i mt*exp(Delta_V/(I_sp*g)) m_i
eq7 C_T C_D + c_t*m_i c_t
eq3 theta_r lambda*rho/D theta_r
eq0 Y IFOV*r IFOV

group1.eq13
NL: Newton Converged in 4 iterations

group1.eq12


C:\Users\johan\Miniconda2\envs\mdo\lib\site-packages\autograd\numpy\numpy_vjps.py:53: RuntimeWarning: overflow encountered in square
  lambda ans, x, y : unbroadcast_f(y, lambda g: - g * x / y**2))


RuntimeError: Singular entry found in 'group1.eq12' <class Impcomp> for row associated with state/residual 'h' ('group1.eq12.h') index 0.

In [118]:
#prob.set_val('lambda', 2.1e-1)
#prob.run_model()

In [119]:
model.list_outputs();

33 Explicit Output(s) in 'model'

varname         value           
--------------  ----------------
inp
  k             [1.381e-23]     
  c             [2.998e+08]     
  Y             [0.22199317]    
  Delta_lambda  [2.e-06]        
  Delta_V       [0.87073231]    
  r             [9.257e+19]     
  C_T           [1000000.]      
  h             [6.626e-34]     
  SNR           [10.]           
  N_r           [25.]           
  lambda        [5.e-07]        
  tau_0         [0.75]          
  tau           [1.]            
  d             [3.e-05]        
  H_i           [0.20671916]    
  rho           [1.22]          
  alpha         [50000.]        
  I_sp          [450.]          
  Q             [1.1]           
  T             [5785.]         
  g             [9.8]           
  m_s           [1175.]         
  R             [6.96e+08]      
group1
  eq12
    N_p         [5.20316581e+17]
  eq8
    H_lambda    [8.34693659e+13]
  eq9
    L           [2.02169482e+25]
  eq1
    f 

In [120]:
out = get_outputs(eqs, model)
easytoread = {key: '{:.2f~}'.format((val*key.varunit).to_compact()) for key,val in out.items()}

In [121]:
easytoread

{k: '13.81 yJ / K',
 Y: '221.99 mm',
 c: '299.80 Mm / s',
 Delta_lambda: '2.00 µm',
 m_i: '368035697565.58 Yg',
 r: '92.57 Em',
 Delta_V: '870.73 mm / s',
 C_T: '1.00 MUSD',
 h: '0.00 s * yJ',
 T_i: '444.09 as',
 N_p: '520316581261001664.00',
 SNR: '10.00',
 N_r: '25.00',
 lambda: '500.00 nm',
 Q_E: '-1.00',
 D: '559.61 Tm',
 tau_0: '0.75',
 C_D: '8762327838955676884795392.00 YUSD',
 theta_r: '1.09 zrad',
 P_in: '580.27 TW',
 H_lambda: '83.47 TW / m ** 3',
 tau: '1.00',
 d: '30.00 µm',
 IFOV: '0.00 rad',
 H_i: '206.72 mJ',
 rho: '1.22',
 alpha: '50.00 kUSD / m ** 3',
 I_sp: '450.00 s',
 Q: '1.10',
 T: '5.79 kK',
 g: '9.80 m / s ** 2',
 c_t: '-23.81 TUSD / g',
 L: '20.22 YW / sr',
 m_s: '1.18 Mg / m ** 2',
 mt: '367963037942.65 Yg',
 f: '12.51 Pm',
 R: '696.00 Mm'}

## Thomas v2

In [9]:
original_ins = getdofs(eqs)

In [10]:
original_ins

{Delta_V,
 Delta_lambda,
 IFOV,
 I_sp,
 N_r,
 Q,
 Q_E,
 R,
 T,
 T_i,
 alpha,
 c,
 c_t,
 d,
 g,
 h,
 k,
 lambda,
 m_s,
 r,
 rho,
 tau,
 tau_0}

In [11]:
force_input = [SNR, CT]
not_input = [IFOV, Ti] #what happens when these are not specified? we might then have freedom of chosing them

In [12]:
new_ins = original_ins.union(force_input) - set(not_input)

In [13]:
eqvcst = {key: {elt for elt in outsetposs if not elt in new_ins}
        for key, outsetposs in eqv.items()}
G, edges_original = bipartite_repr(eqvcst)
vrs = getallvars(eqs)
eqns = eqs.keys()
edges = invert_edges(edges_original)

In [14]:
eqvcst

{0: {IFOV, Y},
 1: {Y, f},
 2: {D, f},
 3: {D, theta_r},
 4: {D, mt},
 5: {m_i, mt},
 6: {C_D, D},
 7: {C_D, m_i},
 8: {H_lambda},
 9: {H_lambda, L},
 10: {D, L, P_in},
 11: {H_i, P_in, T_i},
 12: {H_i, N_p},
 13: {N_p}}

In [25]:
# is the solution to the problem unique anyways, so it makes no sense to optimize?

In [15]:
m = gp.Model('minrevassign')
m.setParam('OutputFlag', False )
x = m.addVars(edges, name="assign", vtype=GRB.BINARY)
# Matching eqs:
fcons = m.addConstrs((x.sum(j,'*') == 1 for j in eqns), name='equations')
varcons = m.addConstrs((var_matched_cons(x, j, not_input) for j in vrs), name='variables')
m.setObjective(gp.quicksum([x[(key, var)] for key,var in dout.items() if var in eqvcst[key]]), GRB.MAXIMIZE)

Using license file C:\Users\johan\gurobi.lic
Academic license - for non-commercial use only - expires 2021-09-23


In [16]:
#m.addConstr(gp.quicksum(x[(r,j)] for r,j in sol)<= n_eqs-1)
m.optimize()

In [17]:
m.objVal

6.0

In [18]:
sol = [(r,j) for (r, j) in edges if x[r,j].x>1e-6]
sol_outset = {key:val for key,val in sol}
sol_outset

{0: IFOV,
 1: Y,
 2: f,
 3: theta_r,
 4: mt,
 5: m_i,
 6: D,
 7: C_D,
 8: H_lambda,
 9: L,
 10: P_in,
 11: T_i,
 12: H_i,
 13: N_p}

In [19]:
d = dict(sol_outset)
new_eqs=reassign(eqs, d)
new_eqv = eqvars(new_eqs)
#new_ins = getdofs(new_eqs)
#d = dout
order = mdf_order(new_eqv, d)
ins = reduce(set.union, new_eqv.values()) - set(d.values())

In [28]:
new_eqs

{0: (IFOV, Y/r),
 1: (Y, d*r/f),
 2: (f, D*d/(2*Q*lambda*rho)),
 3: (theta_r, lambda*rho/D),
 4: (mt, D**2*m_s),
 5: (m_i, mt*exp(Delta_V/(I_sp*g))),
 6: (D, (C_D/alpha)**(1/3)),
 7: (C_D, C_T - c_t*m_i),
 8: (H_lambda, 2*pi*c**2*h/(lambda**5*(exp(c*h/(T*k*lambda)) - 1))),
 9: (L, 0.25*Delta_lambda*H_lambda*R**2*tau),
 10: (P_in, pi*D**2*L/(4*r**2)),
 11: (T_i, H_i/(P_in*tau_0)),
 12: (H_i, N_p*c*h/lambda),
 13: (N_p, SNR*(SNR + sqrt(4*N_r**2 + SNR**2))/(2*Q_E))}

In [27]:
order

[13, (4, 7, 5, 6), 12, 8, 9, 3, 2, 10, 11, 1, 0]

In [20]:
prob = om.Problem()
model = prob.model
buildidpvars(ins, model)
counter = coupled_run(new_eqs, order, (), model, model, d)
prob.setup()
prob.run_model()

eq13 N_p SNR*(SNR + sqrt(4*N_r**2 + SNR**2))/(2*Q_E) N_p
eq4 mt D**2*m_s mt
eq7 C_D C_T - c_t*m_i C_D
eq5 m_i mt*exp(Delta_V/(I_sp*g)) m_i
eq6 D (C_D/alpha)**(1/3) D
eq12 H_i N_p*c*h/lambda H_i
eq8 H_lambda 2*pi*c**2*h/(lambda**5*(exp(c*h/(T*k*lambda)) - 1)) H_lambda
eq9 L 0.25*Delta_lambda*H_lambda*R**2*tau L
eq3 theta_r lambda*rho/D theta_r
eq2 f D*d/(2*Q*lambda*rho) f
eq10 P_in pi*D**2*L/(4*r**2) P_in
eq11 T_i H_i/(P_in*tau_0) T_i
eq1 Y d*r/f Y
eq0 IFOV Y/r IFOV

group1.group2
NL: Newton Converged in 6 iterations


In [21]:
out = get_outputs(eqs, model)
easytoread = {key: '{:.2f~}'.format((val*key.varunit).to_compact()) for key,val in out.items()}

In [24]:
out

{IFOV: 1.592811429113105e-06,
 Delta_lambda: 2e-06,
 P_in: 1.3153497814036748e-15,
 g: 9.8,
 Delta_V: 0.22199317108973948,
 D: 0.8425353908637134,
 k: 1.381e-23,
 d: 3e-05,
 N_r: 25.0,
 c_t: 1163.0,
 mt: 834.0923619355436,
 theta_r: 7.240051950514114e-07,
 r: 9.257e+19,
 m_s: 1175.0,
 Y: 147446553993000.12,
 T: 5785.0,
 Q: 1.1,
 N_p: 609.9019513592784,
 R: 696000000.0,
 h: 6.626e-34,
 Q_E: 0.5,
 I_sp: 450.0,
 H_lambda: 83469365924049.05,
 tau: 1.0,
 rho: 1.22,
 tau_0: 0.75,
 C_D: 29901.750922401738,
 alpha: 50000.0,
 c: 299800000.0,
 T_i: 0.24562386349748425,
 SNR: 10.0,
 L: 2.0216948181732074e+25,
 f: 18.834621256267813,
 lambda: 5e-07,
 C_T: 1000000.0,
 H_i: 2.4231097136920646e-16,
 m_i: 834.1343500237302}